# CxVISION - Asynchronous Inferences

This notebook will guide you in how to deploy the CxVision Model Package from AWS Marketplace through an **asynchronous inference** and the configuration of some services needed to run the solution. 

At the end of this execution, the architecture will look like this one:

<p align="center">
  <img src="./imgs/asynchronous-inference.png">
</p>

## Pre-requisites

1. This notebook renders correctly in Jupyter Notebook interface, so please open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has `AmazonSageMakerFullAccess`.
3. To deploy this ML model successfully, ensure that:
   1. You are able to make AWS Marketplace subscriptions and our IAM role has the following permissions:
      - `aws-marketplace:ViewSubscriptions`
      - `aws-marketplace:Unsubscribe`
      - `aws-marketplace:Subscribe`
4. Create an Amazon S3 Bucket.
5. Create two Amazon SNS topics, for success and failed notifications.


## Contents

1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create the model](#2.-Create-the-model)
3. [Deploy the model](#Deploy-the-model)
4. [Solution configurations](#4.-Solution-configurations)
5. [Perform inferences](#5.-Perform-inferences)
6. [Visualize results](#6.-Visualize-results)
7. [Clean Up your environment](#7.-Clean-Up-your-environment)

### 1. Subscribe to the model package

1. Open the model package page ["Customer experience vision"](https://aws.amazon.com/marketplace/pp/prodview-qraiphwo2242e?ref_=beagle&applicationId=AWS-Sagemaker-Console).
2. Click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review it and click on **"Accept Offer"** if you agree with the EULA, pricing, and support terms.
4. Click on **Continue to configuration button** and then select a **region**. The **Product ARN** will be displayed. **This is the model package ARN that you need to specify while creating a deployable model using Boto3.**


### 2. Create the model

Create the model using the ARN of the CXVision Model Package

In [ ]:
from sagemaker import ModelPackage

sagemaker_role = '<YourSageMakerRole>' # Replace with your IAM Role for Sage Maker
sagemaker_model_package_arn = '<ModelPackageARN>' # Replace this with your subcribed model package ARN

model = ModelPackage(role=sagemaker_role, 
                     model_package_arn=sagemaker_model_package_arn)

### 3. Deploy the model
The following cell deploys an asynchronous endpoint.

> Please, replace the params according to your case.

In [ ]:
import sagemaker
import uuid

instance_type='<InstanceType>' # Replace with your instance type. Supported instances types: ml.g4dn.xlarge, ml.g4dn.2xlarge, ml.g4dn.4xlarge, ml.g4dn.8xlarge, ml.g4dn.12xlarge
bucket = '<YourBucketName>'    # Replace with your bucket name.
output_key = '<OutputFolderKey>' # Replace with your bucket output folder.
success_topic_arn = '<YourSuccessTopicArn>' # Replace with your success SNS Topic ARN
error_topic_arn = '<YourEuccessTopicArn>'   # Replace with your error SNS Topic ARN

job_name='cxvision'
initial_instance_count=1  
endpoint_name = '{}-{}'.format(job_name,str(uuid.uuid4()))

predictor = model.deploy(
    initial_instance_count=initial_instance_count,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    async_inference_config=sagemaker.async_inference.AsyncInferenceConfig(
        output_path=f"s3://{bucket}/{output_key}",
        max_concurrent_invocations_per_instance=1,
        notification_config={
            'SuccessTopic': success_topic_arn,
            'ErrorTopic': error_topic_arn,
        }
    )
)

### 4. Solution configurations

#### 4.1 Define video areas and zones
Please, follow the instructions in the [Define Areas Notebook](./DefineAreas.ipynb) to create the object areas and zones of the videos you will process with the solution. 

#### 4.2 Provision AWS Resources

It's necessary to create the resources specified in the previous diagram. So, please create the stack using the following CloudFormation template:

 [![Launch Stack](https://cdn.rawgit.com/buildkite/cloudformation-launch-stack-button-svg/master/launch-stack.svg)](https://console.aws.amazon.com/cloudformation/home#/stacks/new?stackName=buildkite&templateURL=https://s3.amazonaws.com/my-great-stack.json)

 > Remember specify as parameter the **endpoint name** (name of the previously deployed endpoint) and the **areas object** (created in the step 3.1 with de Define Areas Notebook)



The following resources are created using the cloudformation template:

- **Lambda function (Enqueue videos)**: this function is invoked through an Amazon S3 trigger when a new video is uploaded to a specific Bucket path. The function is in charge of building the payload of the requests to be processed in the endpoint asynchronously, then each request is queued in the FIFO queue and the endpoint is immediately invoked with the first request if the FIFO queue is empty.
- **Lambda function (Invoke endpoint)**: this function is invoked when the asynchronous endpoint completes the execution of a video successfully, sending a success notification to a defined SNS topic. When the function starts, it gets the next request from the FIFO queue and invokes the endpoint with the next video.
- **Lambda function (Cut videos)**: this function fetches a video from an Amazon S3 Bucket, cuts it into fragments of a certain length and finally uploads them to the Amazon S3 Bucket path that will queue the videos in the FIFO queue. The execution of this function is optional, the customer can define its own method of uploading videos to Amazon S3. 
- **S3 Bucket**: an S3 Bucket will be created. In this bucket will be configured all the lambda triggers, also this Bucket will be the input source and the target bucket to processed videos.
- **SQS FIFO Queue**: each of the videos to be processed in sequence by the asynchronous endpoint will be uploaded to this queue.



### 5. Perform inferences

To start the inferences, you must upload the videos to the path of the Amazon S3 bucket defined in the CloudFormation template. To specify the area of the video, you must name the video as follows:

```
{video_name}_area_{area_name}.mp4
```

*  video_name: Name of the video.
*  area_name: Indicates which area that video belongs to. The area must correspond to one of the areas defined in the object created in the step [3.1 Define video areas and zones](#3.1-Define-video-areas-and-zones)

**Example:**
If the video has the following name `example_area_restaurant.mp4`, the zones specified for the **restaurant area** will be taken into account.

Once the videos are uploaded to Amazon S3, they will be queued and subsequently processed by the model.


### 6. Visualize results

The results of each video will be uploaded to the path specified in the payload. The results consist of two files:

1. Processed video with the detections of people, times in each zone and count of these.
2. Text file in the form of logs with the metrics generated during each frame processed in the video. This txt file contains the following properties:

```
Frame 59 - 2022-12-01 13:07:02.127757. Metric_data: total_detections * 6 * video * maki_japan.mp4 * dwell_detections * 3 * dwell_ids * 1,2,4 * dwell_time * 1,1,1 * service_detections * 3 * service_ids * 3,5,6 * service_time * 1,1,1
```


### 7. Clean Up your environment

#### 1. Delete the model and endpoint

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)
model.delete_model()

#### 2. Unsubscribe (optional)

1. Ensure that you do not have a [running model](https://console.aws.amazon.com/sagemaker/home#/models).
2. Go to __Machine Learning__ tab on [Your Software Subscriptions](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust) page.
2. Locate the listing that you want to cancel the subscription for and then choose __Cancel Subscription__.